In [ ]:
#default_exp cli
from nbdev import *
# %nbdev_default_export cli

# cli

> This is a cli module for RemoteConnect

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from fastcore.script import call_parse, Param, bool_arg
from RemoteConnect.core import RemoteCode, RemoteJupyter, RemotePluto, IN_COLAB, mount_drive

## Start Code

In [ ]:
#export
@call_parse
def start_code(port:Param("Port to Start Code", type=int)=10000, 
               password:Param("Password to Start Code", type=str)=None, 
               tunnel:Param("Tunel Type", type=str)='ngrok'):
    "Starts Code Server"
    if IN_COLAB: mount_drive()
    RemoteCode(password=password, port=port, tunnel=tunnel)


## Start Jupyter

In [ ]:
#export
@call_parse
def start_jupyter(port:Param("Port to Start Jupyter", type=int)=9000, 
                  ui:Param("Interface to start", type=str)='notebook',
                  tunnel:Param("Tunel Type", type=str)='ngrok'):
    "Starts Jupyter"
    if IN_COLAB: mount_drive()
    RemoteJupyter(port=port, ui=ui, tunnel=tunnel)

## Start Pluto

In [ ]:
#export
@call_parse
def start_pluto(port:Param("Port to Start Jupyter", type=int)=9000,
                tunnel:Param("Tunel Type", type=str)='ngrok'):
    "Starts Pluto.jl reactive notebook"
    if IN_COLAB: mount_drive()
    RemotePluto(port=port, tunnel=tunnel)